In [56]:
#再帰的に理由を生成させる

#ライブラリの自動インポート
%reload_ext autoreload
%autoreload 2

In [57]:

import pandas as pd
import openai
from tqdm import tqdm
from src.utils.GPT import json_generate
from src.qra_gen import generate_reason_and_predict
import os

In [58]:

openai.api_key =os.environ["OPENAI_API_KEY"]

In [59]:
#laod dataset as dict
"""
csv_path="dataset/BradleyMeltingPointDataset_practice.csv"
df=pd.read_csv(csv_path)
chemical_records=df.to_dict(orient='records')
chemical_records[:1]
"""

[{'mpC': 152.0,
  'name': '3-phthalimidopropionic acid',
  'smiles': 'c1ccc2c(c1)C(=O)N(C2=O)CCC(=O)O',
  'csid': 69310,
  'link': 'http://www.alfa.com/en/GP100W.pgm?DSSTK=L13535',
  'source': 'Alfa Aesar'}]

In [60]:
chemical_record={'mpC': 152.0,
  'name': '3-phthalimidopropionic acid',
  'smiles': 'c1ccc2c(c1)C(=O)N(C2=O)CCC(=O)O',
  'csid': 69310,
  'link': 'http://www.alfa.com/en/GP100W.pgm?DSSTK=L13535',
  'source': 'Alfa Aesar',
  }

In [61]:
model="gpt-4-1106-preview"

In [62]:
system_prompt="""
Provide the quantitative Reason and Prediction so that a scientist, who does not know the melting point, can predict the value.

#Commands
- You must quantitatively consider how the melting point shifts, focusing on each functional groups.
- If Actual value and Prediction differ each other significantly, rethink Reason.
- If Prediction does not contain numbers for each functional group effect, rethink Reason


#Example reason
- Target compound: Toluene
- Basic unit, benzene has a boiling point of 80.
- Methyl group: +30 (due to larger molecular weight)
- Prediction: 110

"""


In [63]:
def gen_prompt(chemical_record,reason="",prediction=""):
    name=chemical_record["name"]
    smiles=chemical_record["smiles"]
    value=chemical_record["mpC"]
    prompt=f"""
#Data
-Name: {name}
-SMILES: {smiles} 
-Actual value: {value}
-Reason: {reason}
-Prediction: {prediction}

#Output (JSON keys)
- Reason, Prediction
"""
    return prompt


In [64]:
import json

#ask gpt
def json_generate(prompt,model="gpt-3.5-turbo-1106"):
    response = openai.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": f"""{prompt}"""
        }  
    ],
    response_format={ "type": "json_object" }
    )

    return (json.loads(response.choices[0].message.content))




In [65]:
import copy
gen_record=copy.deepcopy(chemical_record)
gen_record["Reason"]=""
gen_record["Prediction"]=""

record_history=[]
record_history.append(copy.deepcopy(gen_record))

In [68]:
r=json_generate(
    gen_prompt(gen_record,
               reason=gen_record["Reason"],
               prediction=gen_record["Prediction"]
    ),
    model=model,
)
gen_record.update(r)
record_history.append(copy.deepcopy(gen_record))

In [69]:
record_history

[{'mpC': 152.0,
  'name': '3-phthalimidopropionic acid',
  'smiles': 'c1ccc2c(c1)C(=O)N(C2=O)CCC(=O)O',
  'csid': 69310,
  'link': 'http://www.alfa.com/en/GP100W.pgm?DSSTK=L13535',
  'source': 'Alfa Aesar',
  'Value': 425.0,
  'Reason': '',
  'Prediction': ''},
 {'mpC': 152.0,
  'name': '3-phthalimidopropionic acid',
  'smiles': 'c1ccc2c(c1)C(=O)N(C2=O)CCC(=O)O',
  'csid': 69310,
  'link': 'http://www.alfa.com/en/GP100W.pgm?DSSTK=L13535',
  'source': 'Alfa Aesar',
  'Value': 425.0,
  'Reason': "To predict the melting point of 3-phthalimidopropionic acid, let's consider the effects of its functional groups and structure on its melting point relative to a basic reference compound. The key functional groups and structural features in 3-phthalimidopropionic acid are the phthalimide structure and the propionic acid side chain. The base structure for phthalimide can be phthalic anhydride, which has a melting point of around 131°C. The rigid, planar structure of phthalimide and its ability to